<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/5_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 要用process，不能用thread (要kill才不會影響其他System_UI)

1. 使用stop_event (check_stop) 在 各step function & run_report_function  (直接放在主程式了)
2. 重新thread時， 先stop_event.clear() (解除stop狀態)
3. 添加 kill btn
4. start report ，下方出現狀態列，並且 report btn 變成 disabled (=true)
5. start 按鈕 → 開啟interval。& step-list.style (獨立的callback, 若上個thread還在執行，progress還要變動)
6. 完成後寄信
  kill 按鈕 & report 完成 → 關閉interval。

In [ ]:
# stop_event 與執行緒參考
stop_event = threading.Event()
current_thread = None

In [ ]:
def step5():
    check_stop()
    time.sleep(1)

def step6():
    check_stop()
    time.sleep(1)

# 每一步驟模擬
# def step1():
#     if stop_event.is_set(): return
#     time.sleep(1)

# 檢查是否應停止
def check_stop():
    if stop_event.is_set():
        raise Exception("中止流程")

In [ ]:
# 啟動流程
@app.callback(
    Output("run-btn", "disabled"),
    Output("kill-btn", "disabled"),
    # Output('step-list', 'children'),
    Input("run-btn", "n_clicks"),
    prevent_initial_call=True
)
def start_process(n_clicks):
    global current_thread  # 要重新賦值，要先global
    stop_event.set()  # 通知舊流程停止
    # 確定當前結束
    if current_thread and current_thread.is_alive():
        # 先初始化 progress
        get_initial_progress()
        progress['status'][0]='waiting'
        current_thread.join()

    # 重設狀態 (新執行緒開始前清除 stop 狀態)
    stop_event.clear()
    progress['current_index'] = -1
    progress['step_status'] = ['pending'] * len(step_names)
    progress['step_times'] = [None] * len(step_names)
    progress['error'] = False

    current_thread = threading.Thread(target=run_report_process, daemon=True)
    current_thread.start()
    return True, False

# 停止流程
@app.callback(
    Output("kill-btn", "disabled", allow_duplicate=True),
    Output("run-btn", "disabled", allow_duplicate=True),
    Input("kill-btn", "n_clicks"),
    prevent_initial_call=True
)
def stop_process(n_clicks):
    global current_thread
    stop_event.set()

    if current_thread and current_thread.is_alive():
        # 先初始化 progress
        get_initial_progress()
        progress['status'][0]='waiting'
        current_thread.join()
    return True, False

In [ ]:
# 更新 UI
@app.callback(
    Output("step-list", "children"),
    Output("progress-label", "children"),
    Input("interval", "n_intervals")
)
def update_ui(n):
    elements = []
    for i, step in enumerate(step_names):
        status = progress['step_status'][i]
        color = {'pending': 'gray', 'running': 'orange', 'success': 'green', 'error': 'red'}.get(status, 'gray')
        symbol = {'pending': "□", 'running': "▶", 'success': "✔", 'error': "❌"}.get(status, "□")
        time_spent = f" ({progress['step_times'][i]}秒)" if progress['step_times'][i] else ""
        elements.append(html.Div(f"{symbol} {step}{time_spent}", style={"color": color, "marginBottom": "5px"}))

    current = progress['current_index']
    if progress['error']:
        label = f"流程中斷：{step_names[current]} 發生錯誤"
    elif current == len(step_names) - 1 and progress['step_status'][-1] == 'success':
        label = "流程完成 ✅"
    else:
        label = f"目前進度：{current + 1}/{len(step_names)}"
    return elements, label